Program to query UMLS.

Additional Resources:
https://www.ncbi.nlm.nih.gov/books/NBK9684/
https://documentation.uts.nlm.nih.gov/rest/home.html
https://github.com/HHS/uts-rest-api/tree/master/samples/python

In [9]:
import requests, json
from pyquery import PyQuery as pq
from lxml import etree
from Authentication import *

In [16]:
#################################################################################
# see https://documentation.uts.nlm.nih.gov/rest/search/index.html for full docs
# on the /search endpoint
#################################################################################

apikey = "" #insert your apikey here
version = "current"
uri = "https://uts-ws.nlm.nih.gov"

In [40]:
## search by term
## based on search-terms.py from https://github.com/HHS/uts-rest-api/tree/master/samples/python
## requires Authentication.py from that site

string = "diabetic foot"
#string = "this is not a medical term"
content_endpoint = "/rest/search/"+version
##get at ticket granting ticket for the session
AuthClient = Authentication(apikey)
tgt = AuthClient.gettgt()
pageNumber=0

while True:
    
    ##generate a new service ticket for each page if needed
    ticket = AuthClient.getst(tgt)
    pageNumber += 1
    query = {'string':string,'ticket':ticket, 'pageNumber':pageNumber}
    #query['includeObsolete'] = 'true'
    #query['includeSuppressible'] = 'true'
    #query['returnIdType'] = "sourceConcept"
    query['searchType'] = "exact"
    query['sabs'] = "SNOMEDCT_US"
    r = requests.get(uri+content_endpoint,params=query)
    r.encoding = 'utf-8'
    items  = json.loads(r.text)
    jsonData = items["result"]
    #print (json.dumps(items, indent = 4))
    
    for result in jsonData["results"]:
        try:
             print "ui: " + result["ui"]
        except:
            NameError
        try:
            print "uri: " + result["uri"]
        except:
            NameError
        try:
            print "name: " + result["name"]
        except:
            NameError
        try:
            print "Source Vocabulary: " + result["rootSource"]
        except:
            NameError
        try:
            print "value: " + result['value']
        except:
            NameError
      
        print("\n")
        
    
    ##Either our search returned nothing, or we're at the end
    if jsonData["results"][0]["ui"] == "NONE":
        break
    print("*********")

ui: C0206172
uri: https://uts-ws.nlm.nih.gov/rest/content/2016AA/CUI/C0206172
name: Diabetic Foot
Source Vocabulary: MTH


*********
ui: NONE
name: NO RESULTS




In [23]:
## search by CUI or Code
## based on retreive-cui-or-code.py from https://github.com/HHS/uts-rest-api/tree/master/samples/python
## requires Authentication.py from that site

identifier="C0206172"
AuthClient = Authentication(apikey)
tgt = AuthClient.gettgt()
content_endpoint = "/rest/content/"+str(version)+"/CUI/"+str(identifier)

##ticket is the only parameter needed for this call - paging does not come into play because we're only asking for one Json object
query = {'ticket':AuthClient.getst(tgt)}
r = requests.get(uri+content_endpoint,params=query)
r.encoding = 'utf-8'
items  = json.loads(r.text)
jsonData = items["result"]

##uncomment the print statment if you want the raw json output, or you can just look at the documentation :=)
#https://documentation.uts.nlm.nih.gov/rest/concept/index.html#sample-output
#https://documentation.uts.nlm.nih.gov/rest/source-asserted-identifiers/index.html#sample-output
#print (json.dumps(items, indent = 4))

############################
### Print out fields ####

classType = jsonData["classType"]
name = jsonData["name"]
ui = jsonData["ui"]
AtomCount = jsonData["atomCount"]
Definitions = jsonData["definitions"]
Atoms = jsonData["atoms"]
DefaultPreferredAtom = jsonData["defaultPreferredAtom"]

## print out the shared data elements that are common to both the 'Concept' and 'SourceAtomCluster' class
print ("classType: " + classType)
print ("ui: " + ui)
print ("Name: " + name)
print ("AtomCount: " + str(AtomCount))
print ("Atoms: " + Atoms)
print ("Default Preferred Atom: " + DefaultPreferredAtom)

## These data elements may or may not exist depending on what class ('Concept' or 'SourceAtomCluster') you're dealing with so we check for each one.
try:
   jsonData["definitions"]
   print ("definitions: " + jsonData["definitions"])
   for defs in jsonData['definitions']
except:
      pass

try:
   jsonData["parents"]
   print ("parents: " + jsonData["parents"])
except:
      pass

try:
   jsonData["children"]
   print ("children: " + jsonData["children"])
except:
      pass

try:
   jsonData["relations"]
   print ("relations: " + jsonData["relations"])
except:
      pass

try:
   jsonData["descendants"]
   print ("descendants: " + jsonData["descendants"])
except:
      pass

try:
   jsonData["semanticTypes"]
   print("Semantic Types:")
   for stys in jsonData["semanticTypes"]:
       print("uri: "+ stys["uri"])
       print("name: "+ stys["name"])
      
except:
      pass


classType: Concept
ui: C0206172
Name: Diabetic Foot
AtomCount: 75
Atoms: https://uts-ws.nlm.nih.gov/rest/content/2016AA/CUI/C0206172/atoms
Default Preferred Atom: https://uts-ws.nlm.nih.gov/rest/content/2016AA/CUI/C0206172/atoms/preferred
definitions: https://uts-ws.nlm.nih.gov/rest/content/2016AA/CUI/C0206172/definitions
relations: NONE
Semantic Types:
uri: https://uts-ws.nlm.nih.gov/rest/semantic-network/2016AA/TUI/T047
name: Disease or Syndrome


In [37]:
## get definition given CUI or Code
## based on retreive-cui-or-code.py from https://github.com/HHS/uts-rest-api/tree/master/samples/python
## requires Authentication.py from that site


AuthClient = Authentication(apikey)
tgt = AuthClient.gettgt()
content_endpoint = "/rest/content/"+str(version)+"/CUI/"+str(identifier)+"/definitions"

query = {'ticket':AuthClient.getst(tgt)}
query['sabs'] = "SNOMEDCT_US"
r = requests.get(uri+content_endpoint,params=query)
r.encoding = 'utf-8'
items  = json.loads(r.text)
jsonData = items["result"]
#print (json.dumps(items['result'], indent = 4))

for result in items["result"]:

    try:
        result['value']
        print("definition source: " + result['rootSource'])
        print (result['value'] + "\n")
    except:
        pass


definition source: MSH
Common foot problems in persons with DIABETES MELLITUS, caused by any combination of factors such as DIABETIC NEUROPATHIES; PERIPHERAL VASCULAR DISEASES; and INFECTION. With the loss of sensation and poor circulation, injuries and infections often lead to severe foot ulceration, GANGRENE and AMPUTATION.

definition source: MEDLINEPLUS
<p>If you have <a href='https://www.nlm.nih.gov/medlineplus/diabetes.html'>diabetes</a>, your blood glucose, or <a href='https://www.nlm.nih.gov/medlineplus/bloodsugar.html'>blood sugar</a>, levels are too high. Over time, this can damage your nerves or blood vessels. Nerve damage from diabetes can cause you to lose feeling in your feet. You may not feel a cut, a blister or a sore. Foot injuries such as these can cause ulcers and infections. Serious cases may even lead to amputation. Damage to the blood vessels can also mean that your feet do not get enough blood and oxygen. It is harder for your foot to heal, if you do get a sore o